<a href="https://colab.research.google.com/github/Tobaho/Tobaho.github.io/blob/main/SQL_WRAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#SQL WRAP

I paired up with a classmate(Brenden)to work together on this module. Brenden chose the question "what council district in austin had the most, and longest average bikerides, and how does this correlate with the location of stores in the area".

To solve this question, I will be working with the "Austin bike share data set"

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [9]:
%%bigquery --project tobahodataviz
SELECT *
FROM `bigquery-public-data.austin_bikeshare.bikeshare_trips`
LIMIT 10000

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,13948351,Local365,315,2017-03-07 12:37:33+00:00,1001,Main Office,1001,Main Office,198
1,23248112,Local365,17449,2020-11-27 08:45:21+00:00,3685,9th/Henderson,3685,9th/Henderson,27
2,23304535,Local31,19688,2020-12-12 15:46:14+00:00,4052,Rosewood/Angelina,4057,6th/Chalmers,4
3,23358731,Local31,18673,2020-12-28 16:53:04+00:00,3791,Lake Austin/Enfield,4050,5th/Campbell,36
4,23344637,Pay-as-you-ride,19821,2020-12-25 17:59:12+00:00,3685,9th/Henderson,3685,9th/Henderson,71
...,...,...,...,...,...,...,...,...,...
9995,14951108,Walk Up,182,2017-05-28 11:07:15+00:00,3635,13th & San Antonio,3635,13th & San Antonio,46
9996,14871392,Walk Up,141,2017-05-21 02:53:02+00:00,1007,Lavaca & 6th,3635,13th & San Antonio,2218
9997,14816365,Weekender,670,2017-05-15 21:49:28+00:00,3635,13th & San Antonio,3635,13th & San Antonio,0
9998,15377072,Walk Up,976,2017-07-12 16:56:10+00:00,3635,13th & San Antonio,3635,13th & San Antonio,47


I had to increase the limit, so i can see the subscriber type for the districts. 

In [4]:
%%bigquery --project tobahodataviz

with q as (
  with table as (
    SELECT *
    FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
    LEFT JOIN bigquery-public-data.austin_bikeshare.bikeshare_stations
    ON bikeshare_trips.start_station_id = bikeshare_stations.station_id
  )
  SELECT trip_id AS Trip, station_id AS Station, council_district AS District, subscriber_type AS Type, start_time AS Date, duration_minutes AS DurationMin
  FROM table
)
SELECT District, COUNT(District) AS Frequency, AVG(DurationMin) AS AveDurationMin
FROM q
GROUP BY District
Order BY Frequency DESC

,District,Frequency,AveDurationMin
0,9.0,1056038,28.999327
1,3.0,100457,36.137452
2,1.0,100315,32.689119
3,5.0,88340,37.980269
4,8.0,51006,42.587637
5,10.0,4855,47.403913
6,NaN,0,29.120715


Since we are looking for the district that has the longest average bike rides. This table shows us that district 10 had the highest average duration of minutes and the least frequency. While district 9 had the lowest average duration of minutes and thehighest frequency. 

From this table, we can also deduce the higher the frequency of the district, the lower the average duration of minutes. Why?

Brenden made use of the UT students sub category(location of schools), but I decided to make use of the 24 hours kisosk category(location of stores). I chose this because since the kisosk is being operated at every hour of the day there's a high possibility that this kisosk might be located in district 9.

 I believe that distict 9 is located or probably closer to the center of the city because it takes lesser average duration of minutes(from the table above) for people in district 9 to get to the center of the city. More reason why there is a higher frequency(higher population) in district 9. 

We can not say for a fact that the conclusion deduced above is 100% right, so i decided to group it by "district" and order by "24 hour kisosk"

In [19]:
%%bigquery --project tobahodataviz

with w as (
  with q as (
    with table as (
      SELECT *
      FROM bigquery-public-data.austin_bikeshare.bikeshare_trips
      LEFT JOIN bigquery-public-data.austin_bikeshare.bikeshare_stations
      ON bikeshare_trips.start_station_id = bikeshare_stations.station_id
    )
    SELECT trip_id AS Trip, station_id AS Station, council_district AS District, subscriber_type AS Type, start_time AS Date, duration_minutes AS DurationMin, 
    FROM table
  )
  SELECT District, Type
  FROM q
  WHERE Type = "24-Hour Kiosk (Austin B-cycle)"
)
SELECT District, COUNT(District) AS Twentyfourhourkisosk, 
FROM w 
GROUP BY District
ORDER BY Twentyfourhourkisosk DESC

,District,Twentyfourhourkisosk
0,9.0,72849
1,1.0,10758
2,5.0,7546
3,3.0,5287
4,8.0,2531
5,NaN,0


We can see from the table above that district 9 had the highest number of people in the 24 hour kisosk. 

This wrap up was quite challenging!!